In [ ]:
using HerbGrammar, HerbData, HerbSearch, HerbInterpret, HerbBenchmarks

import HerbInterpret.interpret

The original benchmark uses a wide set of instructions, which are divided into subgroups. The instructions were originally written in Clojure and can be found [here](https://github.com/thelmuth/Clojush/tree/psb2-v1.0/src/clojush/instructions). 

We translated these to Julia functions using ChatGPT with the prompt: "Can you give me julia functions for the following clojure definitions:". Each of the instruction sets can be found in the `grammars` directory. We create a `cfgrammar` for each set, so they can be imported individually. 

In [ ]:
include("grammars/grammar_char.jl")
include("grammars/grammar_gtm.jl")
include("grammars/grammar_bool.jl")
include("grammars/grammar_numbers.jl")
include("grammars/grammar_string.jl")
include("grammars/grammar_random.jl")
include("grammars/grammar_input_output.jl")

In [ ]:
g_random = @cfgrammar begin
    Start = State
    State = make_stacks
    State = boolean_rand(State)
    State = float_rand(State)
    State = integer_rand(State)
    State = string_rand(State)
    State = code_rand(State)
    State = code_rand_atom(State)
    State = char_rand(State)
    State = output1(:output1, State)
    State = output2(:output1, State)
end

g_char = @cfgrammar begin
    State = make_stacks()
    State = char_allfromstring(State)
    State = char_fromfloat(State)
    State = char_fromfloat(State)
    State = char_isletter(State)
    State = char_isdigit(State)
    State = char_iswhitespace(State)
    State = char_lowercase(State)
    State = char_uppercase(State)
end

g_gtm = @cfgrammar begin
    State = make_stacks()
    State = init_gtm(State)
    State = ensure_instruction_map(State)
    State = load_track(State)
    State = dump_track(State)
    State = trace(State)
    State = gtm_left(State)
    State = gtm_right(State)
    State = gtm_inc_delay(State)
    State = gtm_dec_delay(State)
    State = gtm_dub1(State)
end

g_bool = @cfgrammar begin
    State = make_stacks()
    State = boolean_and(State)
    State = boolean_or(State)
    State = boolean_xor(State)
    State = boolean_not(State)
    State = boolean_invert_first_then_and(State)
    State = boolean_invert_second_then_and(State)
    State = boolean_fromfloat(State)
    State = boolean_fromfloat(State)
end

g_integer = @cfgrammar begin
    State = make_stacks()
    State = integer_add(State)
    State = integer_sub(State)
    State = integer_mult(State)
    State = integer_div(State)
    State = integer_mod(State)
    State = integer_lt(State)
    State = integer_gt(State)
    State = integer_lte(State)
    State = integer_gte(State)
    State = integer_fromboolean(State)
    State = integer_fromfloat(State)
    State = integer_fromstring(State)
    State = integer_fromchar(State)
    State = integer_min(State)
    State = integer_max(State)
    State = integer_inc(State)
    State = integer_dec(State)
    State = integer_abs(State)
    State = integer_negate(State)
    State = integer_pow(State)
end

g_float = @cfgrammar begin
    State = make_stacks()
    State = float_add(State)
    State = float_sub(State)
    State = float_mult(State)
    State = float_div(State)
    State = float_mod(State)
    State = float_lt(State)
    State = float_gt(State)
    State = float_lte(State)
    State = float_gte(State)
    State = float_fromboolean(State)
    State = float_fromfloat(State)
    State = float_fromstring(State)
    State = float_fromchar(State)
    State = float_min(State)
    State = float_max(State)
    State = float_inc(State)
    State = float_dec(State)
    State = float_abs(State)
    State = float_negate(State)
    State = float_pow(State)
    State = float_arctan(State)
    State = float_arccos(State)
    State = float_arcsin(State)
    State = float_floor(State)
    State = float_ceiling(State)
    State = float_log10(State)
    State = float_log2(State)
    State = float_square(State)
    State = float_sqrt(State)
    State = float_tan(State)
    State = float_cos(State)
    State = float_sin(State)
end

g_string = @cfgrammar begin
    Start = State
    State = make_stacks()
    State = string_frominteger(State)
    State = string_fromfloat(State)
    State = string_fromchar(State)
    State = string_fromboolean(State)
    State = string_concat(State)
    State = string_conjchar(State)
    State = string_take(State)
    State = string_substring(State)
    State = string_first(State)
    State = string_last(State)
    State = string_nth(State)
    State = string_rest(State)
    State = string_butlast(State)
    State = string_length(State)
    State = string_reverse(State)
    State = string_parse_to_chars(State)
    State = string_split(State)
    State = string_emptystring(State)
    State = string_containschar(State)
    State = string_indexofchar(State)
    State = string_occurrencesofchar(State)
    State = string_replace(State)
    State = string_replacefirst(State)
    State = string_replacechar(State)
    State = string_replacefirstchar(State)
    State = string_removechar(State)
    State = string_setchar(State)
    State = string_capitalize(State)
    State = string_uppercase(State)
    State = string_lowercase(State)
    State = exec_string_iterate(State)
    State = string_sort(State)
    State = string_includes(State)
    State = string_indexof(State)
end;


g_in_out = @cfgrammar begin
    Start = State
    State = make_stacks
    # State = print_integer(State)
    # State = print_float(State)
    # State = print_code(State)
    # State = print_bool(State)
    # State = print_string(State)
    State = print_char(State)
    # State = print_vector_integer(State)
    # State = print_vector_float(State)
    # State = print_vector_boolean(State)
    # State = print_vector_string(State)
end

function merge_grammar(gs::Vector{ContextFreeGrammar})
    new_grammar = @cfgrammar begin end
    for g in gs
        for i in eachindex(g.rules)
            ex = :($(g.types[i]) = $(g.rules[i]))
            add_rule!(new_grammar, ex)
        end
    end
    return new_grammar
end

The following are utility functions that should be added to the `grammars/grammar_util.jl` eventually. We were in the middle of working on the output functionality at the end of the October hackathon for HerbBenchmarks, so these likely will need to change before the benchmark is complete.

In [ ]:
function make_stacks()
    return Dict(
        :char => [],
        :float => [],
        :integer => [],
        :string => [],
        :boolean => [],
    )
end

# Renamed to custom, print_char already exists in `grammar_input_output.jl`
function custom_print_char(output_id::Symbol, state::Dict)
    """
    Adds the first character in the char stack to the output.

    For example, if the char stack is ['a', 'b', 'c'], then the following
    call will add 'a' to the output with id `:output1`.

    `custom_print_char(:output1, state)`

    Args:
        output_id (Symbol): The output id to add the character to
        state (Dict): The state of the program
    
    Returns:
        Dict: The state of the program
    """
    if length(state[:char]) > 0
        state[output_id] = state[:char][1]
    end
    return state
end

`custom_print_char()` is an example of an output function that we can add to the grammar for the `HerbSearch` to use when evaluating programs. For example, if a certain problem had 2 characters as output, we might modify the grammar in the following way:


First, define 2 output functions, one for each character. The name `custom_print_char` just comes from the (Clojure implementation of PushGP)[https://github.com/thelmuth/Clojush/tree/psb2-v1.0], nothing is actually printed.

```julia
output1 = x -> custom_print_char(:output1, x)
output2 = x -> custom_print_char(:output2, x)
```

Then add them to a grammar...

```julia
g = @cfgrammar begin
    ...
    State = output1 | output2
```

Now we are able to construct programs that use the two functions to add characters to the output. For example, the following block shows a program that loads 2 random characters into outputs 1 and 2.

In [ ]:
g_example = @cfgrammar begin
    Start = State
    State = make_stacks()
    State = char_rand(State)
    State = output1(:output1, State)
    State = output2(:output1, State)
end

expr = :(output2(char_rand(output1(char_rand()))))

function interpret(tab::SymbolTable, ex::Expr)
    if ex.head == :call
        if ex.args[1] in keys(tab)
            if length(ex.args) > 1
                return tab[ex.args[1]](interpret(tab, ex.args[2]))
            else
                return tab[ex.args[1]](tab)
            end
        else
            throw(ArgumentError("Argument $(ex.args[1]) not present in symbol table."))
        end
    else
        throw(Error("Expression type not supported: $(ex.head)"))
    end
end

res = interpret(SymbolTable(g_example), expr)

# Small example

This creates a small example of how to use the grammar. We can merge different different grammars together, each should be annotated by its name (e.g., `Dict{Symbol, Any}(:char => SymbolTable(g_char))`).

Then, we create an input dictionary (e.g., `Dict(:input => "a")`), turn it into a state using `init_gtm` and finally merge it with the grammar dictionaries. 

Finally, we create an example of a program and interpret it. 

In [ ]:
# tab_char = Dict{Symbol, Any}(:char => SymbolTable(g_char))
tab_char = SymbolTable(g_char)
tab_gtm = SymbolTable(g_gtm)
tab = merge(tab_char, tab_gtm)
input = Dict(:input => "a")
input = init_gtm(input)
input = merge(tab, input)
stacks = make_stacks()
input = merge(input, stacks)

# println(input[:])
ex = :(char_isletter()) # should be true

interpret(input, ex)

In [ ]:
include("retrieve_all_tasks.jl")
write_psb2_problems_to_file(["coin-sums"], "edge")
include("datasets/coin-sums/coin_sumsdata.jl");

In [ ]:
tab = merge(SymbolTable(g_gtm), merge(SymbolTable(g_integer), merge(SymbolTable(g_string), SymbolTable(g_char))))
stacks = make_stacks()
tab = merge(tab, stacks)
tab
ex = problem_coin_sums.examples[1]

# res = interpret(tab, ex)
# println("Res: ", res)


In [ ]:
function my_evaluator(tab::SymbolTable, expr::Any, input::Dict)
    println("Evaluating: ", expr)
    # println("Input: ", input)
    # println("Tab: ", tab)
    res = interpret(merge(tab, input), expr)
    println(res)
    return res
end


g = merge_grammar([g_char, g_gtm, g_string, g_integer, g_in_out])
sol = HerbSearch.search(g, problem_coin_sums, :Start, max_depth=1000,
    evaluator=my_evaluator,
    allow_evaluation_errors = false,
)